In [ ]:
"""
Streamlined FGDB to SQL Server Converter
Essential components only - production ready
"""

import geopandas as gpd
import fiona
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

def convert_fgdb_to_sql_server(fgdb_path, server, database, username, password, selected_layers=None):
    """
    Convert FGDB layers to SQL Server spatial tables

    Args:
        fgdb_path: Path to .gdb file
        server: SQL Server address (e.g., '100.103.17.32,1433')
        database: Database name
        username: SQL Server username
        password: SQL Server password
        selected_layers: List of layer names, or None for all layers

    Returns:
        dict: Conversion results summary
    """

    # 1. Setup connection
    connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
    engine = create_engine(connection_string, fast_executemany=True)

    print(f"🎯 Converting FGDB to SQL Server")
    print(f"📁 Source: {fgdb_path}")
    print(f"🎯 Target: {server} -> {database}")

    # 2. Get layers to convert
    available_layers = fiona.listlayers(fgdb_path)
    layers_to_convert = selected_layers if selected_layers else available_layers

    print(f"📋 Converting {len(layers_to_convert)} layers: {', '.join(layers_to_convert)}")

    results = {'successful': [], 'failed': []}

    # 3. Convert each layer
    for i, layer_name in enumerate(layers_to_convert, 1):
        print(f"\n[{i}/{len(layers_to_convert)}] Processing: {layer_name}")

        try:
            # Read FGDB layer
            gdf = gpd.read_file(fgdb_path, layer=layer_name)
            print(f"   📊 {len(gdf)} features")

            # Clean column names for SQL Server
            gdf.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_').replace('(', '').replace(')', '')
                          for col in gdf.columns]

            # Convert geometry to WKT
            if 'geometry' in gdf.columns:
                print(f"   🔧 Converting geometry...")
                gdf['Shape'] = gdf['geometry'].apply(lambda x: x.wkt if x and hasattr(x, 'wkt') else None)
                gdf = gdf.drop('geometry', axis=1)

            # Write to SQL Server (let pandas handle data types)
            print(f"   💾 Writing to SQL Server...")
            with engine.connect() as conn:
                conn.execute(text(f"IF OBJECT_ID('{layer_name}', 'U') IS NOT NULL DROP TABLE {layer_name}"))
                conn.commit()

            gdf.to_sql(name=layer_name, con=engine, if_exists='replace', index=False, chunksize=1000)

            # Convert Shape column to GEOMETRY type and create spatial index
            if 'Shape' in gdf.columns:
                print(f"   🗺️ Creating spatial geometry...")
                with engine.connect() as conn:
                    # Add geometry column
                    conn.execute(text(f"ALTER TABLE {layer_name} ADD Shape_Geom GEOMETRY"))

                    # Convert WKT to geometry
                    conn.execute(text(f"""
                        UPDATE {layer_name}
                        SET Shape_Geom = geometry::STGeomFromText(Shape, 4326)
                        WHERE Shape IS NOT NULL
                    """))

                    # Replace text column with geometry column
                    conn.execute(text(f"ALTER TABLE {layer_name} DROP COLUMN Shape"))
                    conn.execute(text(f"EXEC sp_rename '{layer_name}.Shape_Geom', 'Shape', 'COLUMN'"))

                    # Add primary key (required for spatial index)
                    conn.execute(text(f"ALTER TABLE {layer_name} ADD ID INT IDENTITY(1,1) PRIMARY KEY"))

                    # Create spatial index
                    conn.execute(text(f"""
                        CREATE SPATIAL INDEX SIDX_{layer_name}_Shape
                        ON {layer_name}(Shape)
                        USING GEOMETRY_GRID
                        WITH (BOUNDING_BOX = (-180, -90, 180, 90))
                    """))

                    conn.commit()

            print(f"   ✅ Success: {layer_name}")
            results['successful'].append(layer_name)

        except Exception as e:
            print(f"   ❌ Failed: {layer_name} - {e}")
            results['failed'].append(layer_name)

    # Summary
    print(f"\n📊 CONVERSION SUMMARY:")
    print(f"✅ Successful: {len(results['successful'])}")
    print(f"❌ Failed: {len(results['failed'])}")

    if results['successful']:
        print(f"\n🎉 Converted layers: {', '.join(results['successful'])}")
        print(f"Test with: SELECT COUNT(*) FROM {results['successful'][0]};")

    return results

def validate_conversion(server, database, username, password, expected_layers):
    """
    Quick validation of converted data

    Returns:
        dict: Validation results
    """
    print(f"\n🔍 Validating conversion...")

    connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
    engine = create_engine(connection_string)

    validation_results = {}

    for layer in expected_layers:
        try:
            # Get basic stats
            with engine.connect() as conn:
                result = conn.execute(text(f"""
                    SELECT
                        COUNT(*) as total_records,
                        COUNT(Shape) as records_with_geometry,
                        SUM(CASE WHEN Shape.STIsValid() = 1 THEN 1 ELSE 0 END) as valid_geometries
                    FROM {layer}
                """)).fetchone()

                validation_results[layer] = {
                    'total_records': result[0],
                    'records_with_geometry': result[1],
                    'valid_geometries': result[2],
                    'status': 'PASS' if result[0] == result[1] == result[2] else 'ISSUES'
                }

            print(f"   ✅ {layer}: {result[0]} records, {result[2]} valid geometries")

        except Exception as e:
            validation_results[layer] = {'error': str(e), 'status': 'ERROR'}
            print(f"   ❌ {layer}: Error - {e}")

    return validation_results

# Example usage
if __name__ == "__main__":
    # Configuration
    CONFIG = {
        'fgdb_path': r'Z:\Users\brendanhall\GitHub\General_Code\ATFS\FGDB to SQL Server Conversion\esri_ref_data.gdb\esri_ref_data.gdb',
        'server': '100.103.17.32,1433',
        'database': 'SpatialTest',
        'username': 'dbeaver',
        'password': 'dbeaver',
        'selected_layers': None  # None = all layers, or ['states', 'fedlandp'] for specific
    }

    # Run conversion
    results = convert_fgdb_to_sql_server(**CONFIG)

    # Validate results
    if results['successful']:
        validation = validate_conversion(
            CONFIG['server'],
            CONFIG['database'],
            CONFIG['username'],
            CONFIG['password'],
            results['successful']
        )

        print(f"\n🏆 Conversion complete! {len(results['successful'])} layers ready for use.")